#### Imports

In [13]:
import cv2
import os
from pathlib import Path
from omegaconf import OmegaConf

### Se toman las 4 carpetas (cable, capsule, screw y transistor) y se separa su información de testing training y se juntan en un solo dataset, igualmente guardando las etiquetas y se setea el tamaño de cada imagen en 128x128 como se indica en el documento

In [ ]:
# Configuración
DATASETS = ['cable', 'capsule', 'screw', 'transistor']
BASE_PATH = Path('../TareaAutoEncoders')
OUTPUT_PATH = BASE_PATH / 'DATASET_128x128'
IMAGE_SIZE = (128, 128)

# Crear estructura de salida (carpetas planas)
for split in ['train', 'test', 'ground_truth']:
    (OUTPUT_PATH / split).mkdir(parents=True, exist_ok=True)

def process_and_save(src_path: Path, dest_dir: Path, prefix: str, is_mask=False):
    """Lee, redimensiona y guarda. Si is_mask usa INTER_NEAREST."""
    try:
        img = cv2.imread(str(src_path), cv2.IMREAD_UNCHANGED)
        if img is None:
            print(f"⚠️ No se pudo leer: {src_path}")
            return False
        interp = cv2.INTER_NEAREST if is_mask else cv2.INTER_AREA
        resized = cv2.resize(img, IMAGE_SIZE, interpolation=interp)
        dest = dest_dir / f"{prefix}_{src_path.stem}{src_path.suffix}"
        cv2.imwrite(str(dest), resized)
        return True
    except Exception as e:
        print(f"Error con {src_path}: {e}")
        return False

# Procesar datasets
for dataset in DATASETS:
    base = BASE_PATH / dataset

    # train -> normalmente sólo 'good' en estos datasets
    train_dir = base / 'train'
    if train_dir.exists():
        for cls in train_dir.iterdir():
            if not cls.is_dir(): 
                continue
            for img in cls.glob('*.*'):
                prefix = f"{dataset}_train_{cls.name}"
                process_and_save(img, OUTPUT_PATH / 'train', prefix, is_mask=False)

    # test -> incluir good y defectos
    test_dir = base / 'test'
    if test_dir.exists():
        for cls in test_dir.iterdir():
            if not cls.is_dir():
                continue
            for img in cls.glob('*.*'):
                prefix = f"{dataset}_test_{cls.name}"
                process_and_save(img, OUTPUT_PATH / 'test', prefix, is_mask=False)

    # ground_truth -> máscaras (usar nearest)
    gt_dir = base / 'ground_truth'
    if gt_dir.exists():
        for cls in gt_dir.iterdir():
            if not cls.is_dir():
                continue
            for img in cls.glob('*.*'):
                prefix = f"{dataset}_gt_{cls.name}"
                process_and_save(img, OUTPUT_PATH / 'ground_truth', prefix, is_mask=True)


## Configuración de los archivos Hydra

In [17]:
# Crear estructura base
conf_path = Path("conf")
conf_path.mkdir(exist_ok=True)

print("Directorio conf/ creado")

Directorio conf/ creado


In [20]:
# Celda 2: Crear carpetas necesarias
subdirs = ["model", "trainer", "logger", "loss", "optimizer"]
for subdir in subdirs:
    (conf_path / subdir).mkdir(exist_ok=True)

print("✅ Subdirectorios creados:")
for subdir in subdirs:
    print(f"   - conf/{subdir}/")

✅ Subdirectorios creados:
   - conf/model/
   - conf/trainer/
   - conf/logger/
   - conf/loss/
   - conf/optimizer/


In [21]:
# Celda: Crear variaciones de configuración para experimentos
# Variación 1: Latent dim pequeño
latent_small_yaml = """name: autoencoder_latent_small
in_channels: 3
hidden_dims: [32, 64, 128, 256]
latent_dim: 128
use_batch_norm: true
dropout_rate: 0.0
"""

with open("conf/model/autoencoder_latent_small.yaml", "w") as f:
    f.write(latent_small_yaml)

# Variación 2: Latent dim grande
latent_large_yaml = """name: autoencoder_latent_large
in_channels: 3
hidden_dims: [32, 64, 128, 256]
latent_dim: 1024
use_batch_norm: true
dropout_rate: 0.0
"""

with open("conf/model/autoencoder_latent_large.yaml", "w") as f:
    f.write(latent_large_yaml)

print("Variaciones de modelo creadas:")
print("   - autoencoder_latent_small (128)")
print("   - autoencoder_latent_large (1024)")

Variaciones de modelo creadas:
   - autoencoder_latent_small (128)
   - autoencoder_latent_large (1024)


In [23]:
# Celda 3: Crear conf/config.yaml (configuración principal)
config_yaml = """defaults:
  - model: autoencoder
  - trainer: default
  - logger: wandb
  - loss: l2
  - optimizer: adam_mid

seed: 42

data:
  data_dir: 'DATASET_128x128'
  image_size: 128
  batch_size: 32
  num_workers: 2
  validation_split: 0.15
  test_split: 0.15

callbacks:
  monitor: "val/loss"
  mode: "min"
  filename: "{epoch:02d}-{val/loss:.4f}"
  save_top_k: 3

experiment:
  name: "default_experiment"
  description: "Default autoencoder experiment"
"""

with open("conf/config.yaml", "w") as f:
    f.write(config_yaml)

print("conf/config.yaml creado")

conf/config.yaml creado


In [24]:
# Celda 4: Crear modelos - conf/model/autoencoder.yaml
autoencoder_yaml = """name: autoencoder
in_channels: 3
hidden_dims: [32, 64, 128, 256]
latent_dim: 512
use_batch_norm: true
dropout_rate: 0.0
"""

with open("conf/model/autoencoder.yaml", "w") as f:
    f.write(autoencoder_yaml)

print("conf/model/autoencoder.yaml creado")

conf/model/autoencoder.yaml creado


In [25]:
# Celda 5: Crear modelos - conf/model/unet.yaml
unet_yaml = """name: unet
in_channels: 3
base_channels: 32
depth: 4
use_batch_norm: true
dropout_rate: 0.0
"""

with open("conf/model/unet.yaml", "w") as f:
    f.write(unet_yaml)

print("conf/model/unet.yaml creado")

conf/model/unet.yaml creado


In [26]:
# Celda 6: Variaciones de autoencoder con latent_dim pequeño
autoencoder_small_yaml = """name: autoencoder_small
in_channels: 3
hidden_dims: [32, 64, 128]
latent_dim: 128
use_batch_norm: true
dropout_rate: 0.0
"""

with open("conf/model/autoencoder_small.yaml", "w") as f:
    f.write(autoencoder_small_yaml)

print("conf/model/autoencoder_small.yaml creado (latent_dim: 128)")

conf/model/autoencoder_small.yaml creado (latent_dim: 128)


In [27]:
# Celda 7: Variaciones de autoencoder con latent_dim grande
autoencoder_large_yaml = """name: autoencoder_large
in_channels: 3
hidden_dims: [32, 64, 128, 256, 512]
latent_dim: 1024
use_batch_norm: true
dropout_rate: 0.1
"""

with open("conf/model/autoencoder_large.yaml", "w") as f:
    f.write(autoencoder_large_yaml)

print("conf/model/autoencoder_large.yaml creado (latent_dim: 1024)")

conf/model/autoencoder_large.yaml creado (latent_dim: 1024)


In [28]:
# Celda 8: Funciones de pérdida - L1
l1_yaml = """name: l1
type: L1Loss
weight: 1.0
"""

with open("conf/loss/l1.yaml", "w") as f:
    f.write(l1_yaml)

print("conf/loss/l1.yaml creado")

conf/loss/l1.yaml creado


In [29]:
# Celda 9: Funciones de pérdida - L2 (MSE)
l2_yaml = """name: l2
type: MSELoss
weight: 1.0
"""

with open("conf/loss/l2.yaml", "w") as f:
    f.write(l2_yaml)

print("conf/loss/l2.yaml creado")

conf/loss/l2.yaml creado


In [30]:
# Celda 10: Funciones de pérdida - SSIM
ssim_yaml = """name: ssim
type: SSIMLoss
weight: 1.0
window_size: 11
sigma: 1.5
data_range: 1.0
"""

with open("conf/loss/ssim.yaml", "w") as f:
    f.write(ssim_yaml)

print("conf/loss/ssim.yaml creado")

conf/loss/ssim.yaml creado


In [31]:
# Celda 11: Funciones de pérdida - SSIM + L1
ssim_l1_yaml = """name: ssim_l1
type: SSIMLoss_L1
weight_ssim: 0.5
weight_l1: 0.5
window_size: 11
sigma: 1.5
data_range: 1.0
"""

with open("conf/loss/ssim_l1.yaml", "w") as f:
    f.write(ssim_l1_yaml)

print("conf/loss/ssim_l1.yaml creado")

conf/loss/ssim_l1.yaml creado


In [32]:
# Celda 12: Trainer - conf/trainer/default.yaml
trainer_yaml = """max_epochs: 20
gpus: 1
precision: 32
deterministic: true
check_val_every_n_epoch: 1
log_every_n_steps: 10
enable_model_summary: true
gradient_clip_val: 0.0
enable_progress_bar: true
"""

with open("conf/trainer/default.yaml", "w") as f:
    f.write(trainer_yaml)

print("conf/trainer/default.yaml creado")

conf/trainer/default.yaml creado


In [33]:
# Celda 13: Logger - conf/logger/wandb.yaml
wandb_yaml = """project: ae_experiments
entity: null
log_model: false
offline: false
tags: []
"""

with open("conf/logger/wandb.yaml", "w") as f:
    f.write(wandb_yaml)

print("conf/logger/wandb.yaml creado")

conf/logger/wandb.yaml creado


In [34]:
# Celda 14: Optimizer - Adam con LR bajo
adam_low_yaml = """name: adam_low
type: Adam
lr: 1e-4
weight_decay: 0.0
betas: [0.9, 0.999]
"""

with open("conf/optimizer/adam_low.yaml", "w") as f:
    f.write(adam_low_yaml)

print("conf/optimizer/adam_low.yaml creado (lr: 1e-4)")

conf/optimizer/adam_low.yaml creado (lr: 1e-4)


In [35]:
# Celda 15: Optimizer - Adam con LR medio
adam_mid_yaml = """name: adam_mid
type: Adam
lr: 1e-3
weight_decay: 0.0
betas: [0.9, 0.999]
"""

with open("conf/optimizer/adam_mid.yaml", "w") as f:
    f.write(adam_mid_yaml)

print("conf/optimizer/adam_mid.yaml creado (lr: 1e-3)")

conf/optimizer/adam_mid.yaml creado (lr: 1e-3)


In [36]:
# Celda 16: Optimizer - Adam con LR alto
adam_high_yaml = """name: adam_high
type: Adam
lr: 5e-3
weight_decay: 1e-5
betas: [0.9, 0.999]
"""

with open("conf/optimizer/adam_high.yaml", "w") as f:
    f.write(adam_high_yaml)

print("conf/optimizer/adam_high.yaml creado (lr: 5e-3)")

conf/optimizer/adam_high.yaml creado (lr: 5e-3)
